## Redes Neurais - Multilayer Perceptron 

#### Aluno: Altiéris Marcelino Peixoto

##### Carregamento das bibliotecas utilizadas

In [2]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
data_types = {
    "wti_variance":"float",
    "wti_skewness":"float",
    "wti_curtosis":"float",
    "image_entropy":"float",
    "class":"int"
}

columns = ["wti_variance", "wti_skewness", "wti_curtosis", "image_entropy","class"]

dataset = pd.read_csv('dados_autent_bancaria.txt',dtype=data_types,names=columns)

##### 1 - Definição do conjunto de treinamento e teste - (80,20)

In [4]:
X = dataset.drop('class', axis=1)
y = dataset['class']

In [5]:
from sklearn.model_selection import StratifiedShuffleSplit    

sss = StratifiedShuffleSplit(train_size=0.90, n_splits=1, test_size=0.10, random_state=0)  

for train_index, test_index in sss.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [6]:
dataset.groupby('class').count()

,wti_variance,wti_skewness,wti_curtosis,image_entropy
class,,,,
0,762,762,762,762
1,610,610,610,610


In [7]:
y_test.groupby(y_test).count()

class
0    77
1    61
Name: class, dtype: int64

##### 2 - Treinamento e otimização de parâmetros de uma rede Neural utilizando GridSearchCV

In [8]:
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [(1,),(2,),(3,),(4,),(5,),(6,)]
        }
       ]

clf = GridSearchCV(
      MLPClassifier(learning_rate='adaptive', 
                    learning_rate_init=1., 
                    early_stopping=True, 
                    shuffle=True
                    ,random_state=42), param_grid, cv=3, n_jobs=-1, scoring='accuracy')

clf.fit(X_train,y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=True, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='adaptive',
       learning_rate_init=1.0, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=42, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'], 'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,), (6,)]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [9]:
print("Best parameters set found on development set:")
print(clf.best_params_)

Best parameters set found on development set:
{'activation': 'logistic', 'hidden_layer_sizes': (4,), 'solver': 'lbfgs'}


##### 3 - Aplicação da Rede Neural treinada para predição do conjunto de teste

In [10]:
y_pred = clf.predict(X_test)

##### 4 - Acurária do classificador

In [11]:
accuracy_score(y_test, y_pred)

1.0

##### 5 - Matriz de confusão

In [12]:
confusion_matrix(y_test, y_pred)

array([[77,  0],
       [ 0, 61]])

In [13]:
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,77,0,77
1,0,61,61
All,77,61,138
